In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('milestones.csv')
df.head()

,cbm,milestones,total milestones,fraction of milestones reached,variant,agent
0,SWE,2,9,0.22,easy,ReAct + SSH
1,SWE,3,9,0.33,easy,ReAct + SSH
2,SWE,9,9,1.00,easy,ReAct + SSH
3,SWE,9,9,1.00,easy,ReAct + SSH
4,SWE,2,9,0.22,easy,ReAct + SSH


In [12]:
# remove unused variants
'''df = df[~((df['cbm'] == 'MLAB') & (df['variant'] == 'medium') & (df['variant'] == 'very hard'))]
df = df[~((df['cbm'] == 'SWE') & (df['variant'] == 'medium') & (df['variant'] == 'very hard'))]
df = df[~((df['cbm'] == 'WMDP') & (df['variant'] == 'medium') & (df['variant'] == 'very hard'))]'''
dummy_cat = pd.CategoricalDtype(['easy', 'medium', 'hard', 'very hard'])
order = [["easy", "hard"], ["easy", "hard"], ["easy", "hard"], ["easy", "medium", "hard", "very hard"]]
print(order)

[['easy', 'hard'], ['easy', 'hard'], ['easy', 'hard'], ['easy', 'medium', 'hard', 'very hard']]


In [13]:
#order = [cat for cat in dummy_cat.categories if df['variant'].str.contains(cat).any()]
df_filtered = df.groupby(['cbm', 'variant']).filter(lambda x: len(x) > 0)
g = sns.catplot(
    data=df_filtered, kind="swarm",
    x="variant", y="fraction of milestones reached", col="cbm", hue="agent", order=order,
    aspect=.5
)
for ax in g.axes.flat:
    title = ax.get_title()
    new_title = title.split('=')[-1].strip()  # Get the part after '=' and remove any whitespace
    ax.set_title(new_title)
g.set_xlabels("")
g.set_ylabels("% milestones (normalized)")
g.figure.supxlabel('Task Variant', fontsize=12)
g.tight_layout()
plt.show()
g.savefig('milestones.png')

TypeError: unhashable type: 'list'

- % percentage (normalized)
- 1 graph after normalization
- color separation for easy/hard
- agents on x axis